In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
# Load the data
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [6]:
# Tokenize the text
max_words = 10000  # Maximum number of words to keep
max_len = 100  # Maximum length of each tweet

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_df['text'])

X_train = tokenizer.texts_to_sequences(train_df['text'])
X_test = tokenizer.texts_to_sequences(test_df['text'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [7]:
# Assuming your target column is named 'target'
y_train = train_df['target']  # This is already 0 and 1, no encoding needed

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [8]:
# Build the model
embedding_dim = 100

model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_len),
    GRU(64, return_sequences=True),
    GRU(32),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 91ms/step - accuracy: 0.6560 - loss: 0.6052 - val_accuracy: 0.7965 - val_loss: 0.4541
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 17s 88ms/step - accuracy: 0.8831 - loss: 0.3099 - val_accuracy: 0.7630 - val_loss: 0.5070
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 21s 89ms/step - accuracy: 0.9269 - loss: 0.2043 - val_accuracy: 0.7623 - val_loss: 0.5691
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 17s 90ms/step - accuracy: 0.9513 - loss: 0.1447 - val_accuracy: 0.7518 - val_loss: 0.6770


Model is overfitting!

In [10]:
# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Create submission file
submission = pd.DataFrame({
    'id': test_df['id'],
    'target': y_pred_binary.flatten()
})
submission.to_csv('submission.csv', index=False)

102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step
